# Get data

In [1]:
from awpy import Demo

# folder of demos
path = ".demos/"

# match one between Faze and Cloud 9 in the Perfect World Shanghai RMR for 4th place
match = "faze-vs-cloud-9-m1-mirage.dem"

# load demo
dem = Demo(path + match)

In [2]:
# other tables from demoparser that could be useful
for event_name, event in dem.events.items():
    print(f"{event_name}: {event.shape[0]} rows x {event.shape[1]} columns")

smokegrenade_expired: 113 rows x 40 columns
bomb_pickup: 58 rows x 36 columns
round_freeze_end: 20 rows x 16 columns
player_jump: 1256 rows x 36 columns
other_death: 85 rows x 47 columns
hltv_fixed: 111 rows x 24 columns
player_footstep: 1056 rows x 36 columns
begin_new_match: 1 rows x 16 columns
chat_message: 8 rows x 37 columns
buytime_ended: 19 rows x 16 columns
player_blind: 216 rows x 58 columns
bomb_planted: 10 rows x 38 columns
round_poststart: 19 rows x 16 columns
bomb_begindefuse: 6 rows x 37 columns
bomb_defused: 3 rows x 38 columns
round_announce_last_round_half: 1 rows x 16 columns
inferno_expire: 80 rows x 40 columns
player_spawn: 190 rows x 36 columns
weapon_reload: 96 rows x 36 columns
player_disconnect: 3 rows x 41 columns
decoy_detonate: 2 rows x 40 columns
player_team: 10 rows x 41 columns
bomb_beginplant: 11 rows x 37 columns
bomb_dropped: 48 rows x 37 columns
smokegrenade_detonate: 117 rows x 40 columns
round_prestart: 19 rows x 16 columns
round_announce_match_point

# Rounds and Outcomes

* total number of rounds played
* win rate of each team (CT and T)
* Rounds where the bomb was planted but not defused

## total rounds

In [3]:
# one row per round so the number of rows is the number of rounds
dem.rounds.shape[0]

19

There were 19 rounds played

## win rate of each team

In [4]:
# labelling which team was CT and T in the first / second half

# obtaining the first team name and which side it was on and assigning values based on that
if list(dem.kills[['victim_team_name', 'victim_team_clan_name']].iloc[0, :])[0] == "CT":

    # first value is CT in this match
    first_half_CT = list(dem.kills[['victim_team_name', 'victim_team_clan_name']].iloc[0, :])[1]

    # assign T based on CT
    first_half_T = dem.kills.loc[dem.kills['victim_team_clan_name'] != first_half_CT]['victim_team_clan_name'].iloc[0,]

    # switch for second half
    second_half_CT, second_half_T = first_half_T, first_half_CT

else:
    # everything should just be flipped in the alternative case

    first_half_T = list(dem.kills[['victim_team_name', 'victim_team_clan_name']].iloc[0, :])[1]

    # assign T based on CT
    first_half_CT = dem.kills.loc[dem.kills['victim_team_clan_name'] != first_half_T]['victim_team_clan_name'].iloc[0,]

    # switch for second half
    second_half_CT, second_half_T = first_half_T, first_half_CT
    

print(f"CT 1st half: {first_half_CT}\n T 1st half: {first_half_T}\nCT 2nd half: {second_half_CT}\n T 2nd half: {second_half_T}")

CT 1st half: FaZe Clan
 T 1st half: Cloud 9
CT 2nd half: Cloud 9
 T 2nd half: FaZe Clan


In [5]:
import pandas as pd
import numpy as np

# assigning team name to round based on half and which in game team won
dem.rounds['winning_team'] = np.select(
    [
        (dem.rounds['round'].between(1, 12, inclusive='both')) & (dem.rounds['winner'] == "CT"), 
        (dem.rounds['round'].between(1, 12, inclusive='both')) & (dem.rounds['winner'] == "T"),
        (dem.rounds['round'].between(13, 24, inclusive='both')) & (dem.rounds['winner'] == "CT"),
        (dem.rounds['round'].between(13, 24, inclusive='both')) & (dem.rounds['winner'] == "T")
    ], 
    [
        first_half_CT, 
        first_half_T,
        second_half_CT,
        second_half_T
        
    ], 
    default='Unknown'
)

# labelling halves
dem.rounds['half'] = np.select(
    [
        (dem.rounds['round'].between(1, 12, inclusive='both')) , 
        (dem.rounds['round'].between(13, 24, inclusive='both')) ,
    ], 
    [
        "First", 
        "Second",
        
    ], 
    default='Overtime'
)


In [6]:
# counting number of round wins each team won per half
summary_series = dem.rounds.groupby(['winning_team', 'winner', 'half'])['round'].count()

# converting to table and sorting into coherent order
sum_table = pd.DataFrame(summary_series).sort_values(['winning_team', 'half'])


# finding win proportion per half
sum_table['proportion'] = np.select(
    [
        sum_table.index.get_level_values('half') == "First",
        sum_table.index.get_level_values('half') == "Second"
    ],
    [
        round(sum_table['round']/12, 4), # would ideal not like this not be hard coded
        round(sum_table['round']/7, 4)
    ],
    default='err'
)

sum_table

round proportion
winning_team winner half                    
Cloud 9      T      First       4     0.3333
             CT     Second      2     0.2857
FaZe Clan    CT     First       8     0.6667
             T      Second      5     0.7143

## Rounds where bomb was planted and not defused

In [7]:
# joining rounds with what the bomb did each round
rounds_bomb_table = dem.rounds.set_index('round').join(dem.bomb.set_index('round'))

# filtering to rounds where the bomb was actually planted
planted_not_defused_rounds = rounds_bomb_table[(rounds_bomb_table['event'] == 'planted')]

# filtering to rounds where bomb was not defused
planted_not_defused_rounds[planted_not_defused_rounds['reason'] != 'bomb_defused'][['winner', 'reason', 'winning_team', 'half', 'site', 'clock']]

,winner,reason,winning_team,half,site,clock
round,,,,,,
4,T,ct_killed,Cloud 9,First,BombsiteA,00:40
9,T,ct_killed,Cloud 9,First,BombsiteA,00:40
10,T,bomb_exploded,Cloud 9,First,BombsiteB,00:40
12,T,ct_killed,Cloud 9,First,BombsiteB,00:40
15,T,ct_killed,FaZe Clan,Second,BombsiteB,00:40
16,T,ct_killed,FaZe Clan,Second,BombsiteB,00:40
18,T,ct_killed,FaZe Clan,Second,BombsiteB,00:40


### Investigating round 10

Why did the bomb explode in round 10?

In [8]:
# number of kills in round 10
dem.kills[dem.kills['round'] == 10][['attacker_name', 'dmg_health', 'dmg_armor', 'attacker_team_name', 'victim_team_name', 'victim_name', 'victim_health', 'victim_armor_value', 'clock', 'ticks_since_bomb_plant']]

,attacker_name,dmg_health,dmg_armor,attacker_team_name,victim_team_name,victim_name,victim_health,victim_armor_value,clock,ticks_since_bomb_plant
66,Ax1Le,27,3,TERRORIST,CT,frozen,15,100,01:38,<NA>
67,rain,175,6,CT,TERRORIST,interz,100,87,00:51,<NA>
68,Ax1Le,27,3,TERRORIST,CT,broky,27,92,00:43,<NA>


Only 3 people were killed total in the round? 2 counter-terrorists and 1 terrorist

#### GIF of round 10

In [9]:
from awpy.plot import gif, PLOT_SETTINGS
from tqdm import tqdm
import os

if not os.path.isfile("de_mirage.gif"):
    frames = []

    for tick in tqdm(dem.ticks[dem.ticks["round"] == 10].tick.values[::128]):
        frame_df = dem.ticks[dem.ticks["tick"] == tick]
        frame_df = frame_df[
            ["X", "Y", "Z", "health", "armor_value", "pitch", "yaw", "team_name", "name"]
        ]

        points = []
        point_settings = []

        for _, row in frame_df.iterrows():
            points.append((row["X"], row["Y"], row["Z"]))

            # Determine team and corresponding settings
            team = "ct" if row["team_name"] == "CT" else "t"
            settings = PLOT_SETTINGS[team].copy()

            # Add additional settings
            settings.update(
                {
                    "hp": row["health"],
                    "armor": row["armor_value"],
                    "direction": (row["pitch"], row["yaw"]),
                    "label": row["name"],
                }
            )

            point_settings.append(settings)

        frames.append({"points": points, "point_settings": point_settings})

    print("Finished processing frames. Creating gif...")
    gif(f"{dem.header['map_name']}", frames, f"{dem.header['map_name']}.gif", duration=100)

Cloud9 (terrorists) got an early kill (Ax1Le -> frozen) in B-site and then fake rotated out. Faze over reacted and rotated to A and mid to control space. Faze was able to kill (rain -> interz) the lurking player outside of A-site and Cloud9 exploded into B-site. The player (broky) in B-site was not able to pick up any kills on Cloud9's entry and so the call seemingly was made to not retake.


Further questions:
* Were the CTs in a save round? It seems like Faze had some early aggression in Apts that did not play out for them?
* Did the CTs have defuse kits?
* How much health did the CTs have?

In [10]:
# filter to round 10, after the bomb was planted, alive players (non empty inventory), and CT players
ct_round_10_data = dem.ticks[(dem.ticks['round'] == 10) & (dem.ticks['is_bomb_planted'] == True) & (dem.ticks['inventory']) & (dem.ticks['team_name'] == "CT")]

In [11]:
# selection of relevant columns
ct_round_10_data.drop_duplicates(subset = "name", keep="first")[['name', 'inventory', 'health', 'armor_value', 'has_defuser', 'has_helmet', 'current_equip_value']]

,name,inventory,health,armor_value,has_defuser,has_helmet,current_equip_value
830962,rain,"[knife_butterfly, Desert Eagle, AK-47, Molotov]",68,96,True,True,5200
830966,ropz,"[knife_karambit, Five-SeveN]",100,100,False,False,1450
830968,karrigan,"[knife_karambit, USP-S, MP9]",75,84,False,False,2400


Very minimal equipment from the CTs: rain had a kit, helmet, but was hit for nearly 30 hp; ropz was at full health but had no helmet nor kit and only a pistol; karrigan had an smg, was already hit, had no kit nor helmet. 

Seems like a good call to not try to retake

In [12]:
# the CTs did not pick up any new weapons after the round ended
dem.events['item_pickup'][(dem.events['item_pickup']['round'] == 10) & (dem.events['item_pickup']['is_bomb_planted'] == True)]

,ct_team_clan_name,ct_team_name,defindex,game_phase,game_time,is_bomb_planted,is_ct_timeout,is_freeze_period,is_match_started,is_technical_timeout,...,user_last_place_name,user_name,user_ping,user_pitch,user_steamid,user_team_clan_name,user_team_name,user_yaw,user_zoom_lvl,round


State of economy beginning of round 10 and round 11

In [13]:
# grab first tick of rounds 10 and 11 to see how much money the CTs have
ct_economy_rounds_10_11 = dem.ticks[(dem.ticks['round'].isin([10, 11])) & (dem.ticks['team_name'] == "CT")].drop_duplicates(subset = ['name', 'round'], keep = 'first')[['name', 'inventory', 'round', 'current_equip_value', 'armor_value', 'has_helmet']]

ct_economy_rounds_10_11

,name,inventory,round,current_equip_value,armor_value,has_helmet
779190,broky,[knife_butterfly],10,1500,100,True
779192,rain,"[knife_butterfly, Desert Eagle, Smoke Grenade,...",10,2600,100,True
779196,ropz,"[knife_karambit, Smoke Grenade]",10,1450,100,False
779197,frozen,"[knife_stiletto, USP-S]",10,2100,100,False
779198,karrigan,"[knife_karambit, USP-S, Smoke Grenade]",10,2400,100,False
874480,broky,"[knife_butterfly, USP-S]",11,200,0,False
874482,rain,"[knife_butterfly, AK-47, Molotov]",11,4700,96,True
874486,ropz,"[knife_karambit, Flashbang]",11,1350,26,False
874487,frozen,"[knife_stiletto, Desert Eagle]",11,700,0,False
874488,karrigan,"[knife_karambit, USP-S]",11,2100,84,False


In [14]:
ct_economy_rounds_10_11.groupby('round')['current_equip_value'].sum()

round
10    10050
11     9050
Name: current_equip_value, dtype: uint32

Look up HLTV common monetary boundaries but fairly sure these are semi-buy to nearly eco rounds

# Round Durations

In [15]:
(60*1 + 55)

115

## What is the average duration of the rounds?

In [16]:
rounds = dem.rounds.copy()

# calculate total number of ticks
# end is when the round winner has been announced. After this there is an additional 7 seconds of gameplay
# freeze_end refers to when players are able to freely move around are not in the buy *only* phase
# evidence of this is found in rounds where the reason for round is 'time_ran_out' and the length of the round is 115 seconds
# round are 1:55 minutes in length which is 115 seconds
rounds['total_ticks'] = rounds['end'] - rounds['freeze_end']

# fairly sure matches are played on 64 tick servers so divide ticks by 64 to get seconds
rounds['seconds'] = round(rounds['total_ticks']/64, 2)

# quartile ranges for number of seconds
qs = list(rounds['seconds'].quantile(q = [0, .25, .5, .75, 1]))

# add average time
qs.append(rounds['seconds'].mean())

# create table 
seconds_five_num_summary = pd.DataFrame(
    qs
).T.rename(mapper = {0: "Min", 1: "1st quartile", 2: "Median", 3: "3rd quartile", 4: "Max", 5: "Average"}, axis = 1)

seconds_five_num_summary

,Min,1st quartile,Median,3rd quartile,Max,Average
0,31.0,76.0,87.44,115.0,135.5,93.389474


50% of rounds lasted a minute and a half or less. The average round lasted just over a minute and a half. Interestingly, pnly 25% of rounds lasted longer than the full length of the round

In [17]:
rounds.sort_values('seconds')

,round,start,freeze_end,end,official_end,winner,reason,bomb_plant,winning_team,half,total_ticks,seconds
16,17,136362,137642,139626,140074,T,ct_killed,<NA>,FaZe Clan,Second,1984,31.0
18,19,152480,156273,160290,160290,T,ct_killed,<NA>,FaZe Clan,Second,4017,62.77
15,16,130384,131664,135914,136362,T,ct_killed,135712,FaZe Clan,Second,4250,66.41
0,1,1016,2712,7392,7840,CT,bomb_defused,5150,FaZe Clan,First,4680,73.12
14,15,123931,125211,129936,130384,T,ct_killed,129852,FaZe Clan,Second,4725,73.83
12,13,108290,109986,114989,115437,CT,bomb_defused,113045,Cloud 9,Second,5003,78.17
8,9,67639,71090,76198,76646,T,ct_killed,74168,Cloud 9,First,5108,79.81
5,6,38801,42528,47703,48151,CT,t_killed,<NA>,FaZe Clan,First,5175,80.86
3,4,22506,23786,29265,29713,T,ct_killed,27115,Cloud 9,First,5479,85.61
2,3,15182,16462,22058,22506,CT,t_killed,<NA>,FaZe Clan,First,5596,87.44


## Which round was the shortest, and which was the longest?

In [18]:
# shortest round
rounds[rounds['seconds'] == rounds['seconds'].min()]

,round,start,freeze_end,end,official_end,winner,reason,bomb_plant,winning_team,half,total_ticks,seconds
16,17,136362,137642,139626,140074,T,ct_killed,<NA>,FaZe Clan,Second,1984,31.0


Round 17 was the shortest round

In [19]:
# longest round
rounds[rounds['seconds'] == rounds['seconds'].max()]

,round,start,freeze_end,end,official_end,winner,reason,bomb_plant,winning_team,half,total_ticks,seconds
7,8,57239,58519,67191,67639,CT,bomb_defused,65499,FaZe Clan,First,8672,135.5


Round 8 was the longest round

# Player Performance

In [20]:
# obtain player round team name information
player_info = dem.kills[['round', 'attacker_name', 'attacker_team_clan_name','attacker_team_name']].copy().drop_duplicates().set_index(['attacker_name', 'round'])


# index is just player name and only column is which team they are on
player_teams = player_info.reset_index().drop_duplicates('attacker_name').set_index('attacker_name')['attacker_team_clan_name'].sort_values()


## Who had the most kills in the match?

In [21]:
kills = dem.kills.copy()

# count the number of rows each player gets in each round
# fill in missing values with 0 kills
# pivot table to make 1 row per round player to allow for filtering based on number of kills
kills_per_round = kills.groupby(['round', 'attacker_name'])['tick'].count().unstack().fillna(0).reset_index().melt(id_vars = "round", value_name = "kills")

player_kills = kills_per_round.groupby('attacker_name')['kills'].sum().reset_index().sort_values('kills', ascending=False).set_index('attacker_name')

player_kills.join(player_teams)

,kills,attacker_team_clan_name
attacker_name,,
rain,21.0,FaZe Clan
HeavyGod,18.0,Cloud 9
ropz,18.0,FaZe Clan
Ax1Le,14.0,Cloud 9
frozen,14.0,FaZe Clan
karrigan,13.0,FaZe Clan
broky,11.0,FaZe Clan
Boombl4,10.0,Cloud 9
interz,9.0,Cloud 9


rain got the most kills

## How many players achieved multi-kills?

In [22]:
# number of multi-kills to look at
num_kills = 2

# sort and set index to allow joining for more context of players
multikill_round_players = kills_per_round.sort_values('round').set_index(['attacker_name', 'round'])

# join information about player context and filter to number of multi-kills
multikills = multikill_round_players.join(player_info)[multikill_round_players['kills'] >= num_kills].reset_index()

# show unique players overall
multikills.drop_duplicates(subset = "attacker_name").reset_index(drop = True)

,attacker_name,round,kills,attacker_team_clan_name,attacker_team_name
0,broky,1,2.0,FaZe Clan,CT
1,Boombl4,1,2.0,Cloud 9,TERRORIST
2,ropz,1,2.0,FaZe Clan,CT
3,rain,2,2.0,FaZe Clan,CT
4,karrigan,3,2.0,FaZe Clan,CT
5,Ax1Le,4,3.0,Cloud 9,TERRORIST
6,frozen,8,2.0,FaZe Clan,CT
7,ICY,8,2.0,Cloud 9,TERRORIST
8,interz,9,2.0,Cloud 9,TERRORIST
9,HeavyGod,13,2.0,Cloud 9,CT


Every player had at least 1 round where they got a multi-kll (CT or T)

In [23]:
# unique players by side
multikills.drop_duplicates(subset = ["attacker_name", 'attacker_team_name']).reset_index(drop = True).sort_values(['attacker_name', 'attacker_team_name'], ascending = [True, False])

,attacker_name,round,kills,attacker_team_clan_name,attacker_team_name
15,karrigan,19,2.0,FaZe Clan,TERRORIST
4,karrigan,3,2.0,FaZe Clan,CT
5,Ax1Le,4,3.0,Cloud 9,TERRORIST
1,Boombl4,1,2.0,Cloud 9,TERRORIST
9,HeavyGod,13,2.0,Cloud 9,CT
7,ICY,8,2.0,Cloud 9,TERRORIST
14,broky,17,2.0,FaZe Clan,TERRORIST
0,broky,1,2.0,FaZe Clan,CT
10,frozen,13,2.0,FaZe Clan,TERRORIST
6,frozen,8,2.0,FaZe Clan,CT


All FaZe members got a multi-kill on both halves. On Cloud 9: Ax1Le, Boombl4, and Icy did not get multi-kills on CT; HeavyGod did not get a multi-kill on T; and interz got a multi-kill on both halves.

#### Who got the most multi-kills?

In [24]:
# group by player name and count the number of rows (rounds with multi-kills) then sort by descending to find most
total_multikills = multikills.groupby(['attacker_name'])['attacker_team_clan_name'].count().reset_index().sort_values('attacker_team_clan_name', ascending=False).rename(mapper = {'attacker_team_clan_name': "number_multi_kills"}, axis = 1).set_index('attacker_name')

total_multikills.join(player_teams)

,number_multi_kills,attacker_team_clan_name
attacker_name,,
rain,7,FaZe Clan
ropz,6,FaZe Clan
HeavyGod,4,Cloud 9
frozen,4,FaZe Clan
karrigan,3,FaZe Clan
Ax1Le,3,Cloud 9
Boombl4,2,Cloud 9
broky,2,FaZe Clan
interz,2,Cloud 9


rain got the most multi-kills

I wonder if getting a multi-kill in a round is a good indicator if you will win that round.

Conversely, how many rounds did a team *lose* after getting a mutli-kill in a round?

# Headshots

## What percentage of kills were headshots

In [25]:
# filter to just kills that were headshots
# count number of filtered rows over total number of kill rows 
round(kills[kills['headshot'] == True].shape[0]/kills.shape[0], 4)

0.5882

## Who had the highest headshot percentage

In [26]:
# same code as before now filtering to just kills that were headshots
# group by the round and player
# count number of rows (kills)
# fill the rest with 0
# pivot to make the kills column filterable
head_shot_kills = kills[kills['headshot'] == True].groupby(['round', 'attacker_name'])['tick'].count().unstack().fillna(0).reset_index().melt(id_vars = "round", value_name = "headshot_kills")

# group by player and get the total number of (headshot) kills
player_kills_headshot = head_shot_kills.groupby('attacker_name')['headshot_kills'].sum().reset_index().sort_values('headshot_kills', ascending=False).set_index('attacker_name')

# join data with total number of kills
player_joined_kill_headshot = player_kills.join(player_kills_headshot)

# create percentage
player_joined_kill_headshot['percentage'] = player_joined_kill_headshot['headshot_kills']/player_joined_kill_headshot['kills']


player_joined_kill_headshot

,kills,headshot_kills,percentage
attacker_name,,,
rain,21.0,18.0,0.857143
HeavyGod,18.0,12.0,0.666667
ropz,18.0,13.0,0.722222
Ax1Le,14.0,8.0,0.571429
frozen,14.0,8.0,0.571429
karrigan,13.0,4.0,0.307692
broky,11.0,2.0,0.181818
Boombl4,10.0,7.0,0.700000
interz,9.0,5.0,0.555556


rain had the highest headshot percentage and highest number of headshot kills

# Weapon Analysis

In [27]:
# using item equip table to get grouping of weapons
weapon_id_table = dem.events['item_equip'][['item', 'weptype']].sort_values('weptype').drop_duplicates().rename(columns = {'item':'weapon'}).set_index('weapon')

# reidentify weapon type numbers to weapon type text
weapon_id_table['weptype'] = np.select(
    [
        weapon_id_table['weptype'] == 0,
        weapon_id_table['weptype'] == 1,
        weapon_id_table['weptype'] == 2,
        weapon_id_table['weptype'] == 3,
        weapon_id_table['weptype'] == 4,
        weapon_id_table['weptype'] == 5,
        weapon_id_table['weptype'] == 6,
        weapon_id_table['weptype'] == 7,
        weapon_id_table['weptype'] == 8,
        weapon_id_table['weptype'] == 9,
    ],
    [
        'knife',
        'pistol',
        'smg',
        'rifle',
        'unknown/not play in this match',
        'sniper',
        'unknown/not play in this match',
        'bomb',
        'unknown/not play in this match',
        'grenade'
    ],
    default='err'
)

weapon_id_table

,weptype
weapon,
knife,knife
hkp2000,pistol
glock,pistol
elite,pistol
deagle,pistol
p250,pistol
fiveseven,pistol
tec9,pistol
mp9,smg


## Which weapon was used the most across all rounds?

In [28]:
inventories = dem.ticks[['inventory', 'name', 'round', 'tick']].copy().reset_index(drop = True)

inventories['inventory'].value_counts()

inventory
[]                                                                 331129
[knife_karambit, Glock-18, AK-47, Flashbang]                        26995
[knife_butterfly, USP-S, M4A1-S]                                    25277
[knife_karambit, USP-S]                                             21708
[knife_butterfly, USP-S]                                            21453
                                                                    ...  
[knife_butterfly, USP-S, Smoke Grenade, Molotov, AWP]                   6
[knife_butterfly, Desert Eagle, AK-47]                                  5
[knife_kukri]                                                           4
[knife_butterfly, Desert Eagle, High Explosive Grenade, M4A1-S]         2
[knife_butterfly, Desert Eagle, MAC-10]                                 1
Name: count, Length: 425, dtype: int64

In [29]:
dem.ticks[(dem.ticks['round'] == 1) & (dem.ticks['name'] == "HeavyGod") & (dem.ticks['tick'] <= 4000)]

,inventory,accuracy_penalty,zoom_lvl,is_bomb_planted,ping,health,has_defuser,has_helmet,flash_duration,last_place_name,...,X,pitch,yaw,Y,Z,game_time,tick,steamid,name,round
27129,[knife_widowmaker],0.0000,NaN,False,0,100,False,False,0.0,TSpawn,...,1136.000000,25.165558,-71.176643,-160.000000,-164.430542,2972.562500,2712,76561198068002993,HeavyGod,1
27139,[knife_widowmaker],0.0000,NaN,False,0,100,False,False,0.0,TSpawn,...,1136.108276,25.165558,-71.176643,-160.317749,-164.432861,2972.578125,2713,76561198068002993,HeavyGod,1
27149,[knife_widowmaker],0.0000,NaN,False,0,100,False,False,0.0,TSpawn,...,1136.291992,25.165558,-71.176643,-160.856995,-164.436890,2972.593750,2714,76561198068002993,HeavyGod,1
27159,[knife_widowmaker],0.0000,NaN,False,0,100,False,False,0.0,TSpawn,...,1136.551392,25.165558,-71.176643,-161.617920,-164.442566,2972.609375,2715,76561198068002993,HeavyGod,1
27169,[knife_widowmaker],0.0000,NaN,False,0,100,False,False,0.0,TSpawn,...,1136.886475,25.165558,-71.176643,-162.600525,-164.449951,2972.625000,2716,76561198068002993,HeavyGod,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39969,"[knife_widowmaker, P250]",0.0091,0.0,False,0,100,False,False,0.0,PalaceInterior,...,493.572266,-0.409241,172.062714,-2331.530273,-39.968750,2992.625000,3996,76561198068002993,HeavyGod,1
39979,"[knife_widowmaker, P250]",0.0091,0.0,False,0,100,False,False,0.0,PalaceInterior,...,493.669373,-0.409241,172.062714,-2330.831543,-39.968750,2992.640625,3997,76561198068002993,HeavyGod,1
39989,"[knife_widowmaker, P250]",0.0091,0.0,False,0,100,False,False,0.0,PalaceInterior,...,493.709137,-0.409241,172.062714,-2330.545410,-39.968750,2992.656250,3998,76561198068002993,HeavyGod,1
39999,"[knife_widowmaker, P250]",0.0091,0.0,False,0,100,False,False,0.0,PalaceInterior,...,493.675842,-0.409241,172.062714,-2330.783691,-39.968750,2992.671875,3999,76561198068002993,HeavyGod,1


In [30]:
# # get ticks at the beginning of each round
# beginning_round_ticks = dem.ticks.drop_duplicates(['name', 'round'], keep = 'first')[['tick']]

# # each round starts with a *seconds* seconds of a buy time 
# # need to wait until it expires to see what equipment players buy
# ticks_per_second = 64
# seconds = 17

# # find ticks at later time
# beginning_round_ticks[f'{seconds}_seconds_later'] = beginning_round_ticks['tick'] + (ticks_per_second*seconds)

# # get unique ticks
# starting_ticks = beginning_round_ticks[f'{seconds}_seconds_later'].drop_duplicates()

# see player information at that time period
player_invs = dem.ticks[dem.ticks['tick'].isin(rounds['freeze_end'])][['tick', 'inventory', 'name', 'current_equip_value', 'round', 'team_clan_name']].sort_values(['round', 'team_clan_name'])

player_invs[player_invs['name'] == "HeavyGod"]

,tick,inventory,name,current_equip_value,round,team_clan_name
27129,2712,[knife_widowmaker],HeavyGod,950,1,Cloud 9
91199,9120,"[knife_widowmaker, Glock-18]",HeavyGod,3000,2,Cloud 9
164619,16462,"[knife_widowmaker, Desert Eagle]",HeavyGod,700,3,Cloud 9
237859,23786,"[knife_widowmaker, Glock-18, AK-47, Smoke Gren...",HeavyGod,5000,4,Cloud 9
309929,30993,"[knife_widowmaker, Glock-18, AK-47, Smoke Gren...",HeavyGod,5000,5,Cloud 9
425279,42528,"[knife_widowmaker, Glock-18, AK-47, Smoke Gren...",HeavyGod,4200,6,Cloud 9
494299,49431,"[knife_widowmaker, Desert Eagle]",HeavyGod,1350,7,Cloud 9
585169,58519,"[knife_widowmaker, Glock-18, AK-47, Smoke Gren...",HeavyGod,5100,8,Cloud 9
710839,71090,"[knife_widowmaker, Glock-18, AK-47, Smoke Gren...",HeavyGod,4400,9,Cloud 9
779199,77926,"[knife_widowmaker, Glock-18, High Explosive Gr...",HeavyGod,5100,10,Cloud 9


## What are the kill counts for each type of weapon (rifle, SMG, pistols, etc)?

In [31]:
def kills_grouping(grouping_types = ['weapon']):
    return pd.DataFrame(dem.kills.groupby(grouping_types)['tick'].count()).rename(mapper = {'tick': 'kills'}, axis = 1).sort_values('kills', ascending=False)


In [32]:
kills_grouping(['weapon'])

,kills
weapon,
ak47,57
m4a1_silencer,26
usp_silencer,10
awp,9
deagle,9
glock,5
mp9,4
famas,3
galilar,3


### Player kills by weapon

In [33]:
kills_grouping(['attacker_name', 'weapon'])[kills_grouping(['attacker_name', 'weapon'])['kills'] > 2]

kills
attacker_name weapon              
rain          ak47              11
Ax1Le         ak47              10
 karrigan     ak47               8
frozen        ak47               8
ropz          m4a1_silencer      7
              ak47               6
HeavyGod      m4a1_silencer      5
broky         awp                5
Boombl4       ak47               4
 karrigan     m4a1_silencer      4
HeavyGod      ak47               4
ICY           awp                4
rain          m4a1_silencer      3
HeavyGod      deagle             3
rain          deagle             3
interz        ak47               3

# Clutching 1vX situations

In [34]:
round_winner = rounds[['round', 'winner', 'reason']].set_index('round')

In [35]:
# detect rounds where 1 player is left alive on each team
# find rounds were 4 teammates died
deaths_per_round = kills_grouping(['round', 'victim_team_name']).sort_index().reset_index().set_index('round').rename(columns = {'victim_team_name':'team', 'kills':'deaths'})

# find winner of that round and confirm that the respective team won that round
clutch_rounds = deaths_per_round[deaths_per_round['deaths'] == 4].join(round_winner).reset_index()

clutch_rounds

,round,team,deaths,winner,reason
0,1,CT,4,CT,bomb_defused
1,4,TERRORIST,4,T,ct_killed
2,7,CT,4,CT,time_ran_out
3,18,TERRORIST,4,T,ct_killed


In [39]:
pd.DataFrame(kills[kills['round'].isin(clutch_rounds['round'])].groupby(['attacker_name', 'round'])['tick'].count())

tick
attacker_name round      
 karrigan     4         1
              7         3
Ax1Le         1         1
              4         3
              7         1
              18        1
Boombl4       1         2
              7         3
              18        1
HeavyGod      1         1
              4         1
              18        1
broky         1         2
              18        1
frozen        1         1
              18        1
interz        4         1
              18        1
rain          4         2
              7         2
              18        1
ropz          1         2
              4         1
              18        2

In [893]:
example_round = {}

kills[kills['round'] == 4][['victim_team_name', 'victim_name']]

,victim_team_name,victim_name
21,TERRORIST,ICY
22,CT,karrigan
23,TERRORIST,HeavyGod
24,CT,frozen
25,TERRORIST,Boombl4
26,CT,broky
27,CT,rain
28,TERRORIST,Ax1Le
29,CT,ropz
